<center><h1>Telco Churn Project</h1></center>

# Table of contents <a name ='toc'></a>
1. [Business Goals](#business_goals)
     1. [Drivers of Churn](#drivers_of_churn)
     2. [Machine Learning Construction](#ml_construction)
2. [Data Acqisition](#data_acquisition)
3. [Preparation](#preparation)
3. [Exploratory Data Analysis](#exp_data_analysis)
4. [Statistical Testing](#stat_testing)
5. [Modeling](#modeling)
6. [Model Evaluation](#model_eval)

## Business Goals <a name="business_goals"></a>
[Table of Contents](#toc)

